# Sphere packing and uncertainty principles

## Sphere packing - upper bounds based on linear programming methods

**Prompt for the search setup**

Act as an expert software developer and optimization specialist specializing in
creating linear combinations of Laguerre polynomials with certain properties.

For a given pair of natural numbers (m, n) we consider the following
construction.

Let $g_k(x) = L_k^\alpha(x)$ where $L_k^\alpha(x)$ denotes the standard
generalized Laguerre polynomial of degree k and order $\alpha = (n / 2) - 1$.

First, select a collection of m positive real numbers z_1, z_2, \dots, z_m.

Then let g(x) be a linear combination of odd degree g_k's up to order $4m + 3$
i.e. a linear sum of g_1, g_3, \dots, g_{4m + 3}. The coefficients of this
linear combination are chosen such that z_1, \dots, z_m are double roots and
g(0) = 0 and g'(0) = 1 (Counting degrees of freedom implies that there is a
unique such linear combination g(x)).

Finally, define $r$ to be the largest positive sign change of the polynomial
g(x).

GOAL:
Your optimization task is, for a given natural numbers m and n, to find a
collection of m positive real numbers z_1, z_2, \dots, z_m such that the
polynomial g(x) has the largest possible value of r.

Specifically, the Python function you have to provide has the following
signature:

def get_roots(m: int = 10, n: int = 8) -> list[float] | np.ndarray:

The function get_roots returns the one-dimensional array z consisting of
the positive real numbers $z_i$ for $i = 1, \dots, m$.
NOTE:
Try to find constructions of roots that are not too spread apart or too large.
The absolute values of the roots z_i should be not larger than 200.
Please keep in mind that a promising starting configuration for m = 11, n = 8 is
given by z = [37.705, 50.285, 62.893, 75.578, 88.454, 101.737, 115.776, 131.035,
148.162, 168.215, 193.766]. Consider using this as a start and inspiration for
further optimization for other m and n.

The score of z is given by the corresponding value of r. To compute this we use
precise fractional arithmetic in sympy.

The exact score function your construction will be evaluated on is given below:

def get_score(m: int, n: int, zs: np.ndarray | list[float]) -> float:
  """Returns the score of the given Laguerre combination."""
  if len(zs) != m:
    return WRONG_Z_SHAPE

  g_fn = find_laguerre_combination(m, n, zs)
  x = sympy.symbols('x')
  dg_fn = sympy.diff(g_fn, x)

  div = sympy.prod([(x - sympy.Rational(z)) ** 2 for z in zs]) * x
  gq_fn = sympy.exquo(g_fn, div)

  g_fn.subs(x, sympy.Rational(0))
  dg_fn.subs(x, sympy.Rational(1))

  for z in zs:
    if g_fn.subs(x, sympy.Rational(z)) != 0:
      return ROOTS_NOT_ENFORCED
    if dg_fn.subs(x, sympy.Rational(z)) != 0:
      return DERIVATIVES_NOT_ENFORCED

  real_roots = sympy.real_roots(gq_fn, x)
  if not real_roots:
    return NO_SIGN_CHANGES

  approx_roots = list()
  largest_sign_change = 0

  for root in real_roots:
    approx_root = root.eval_rational(n=200)
    approx_root_p = approx_root + sympy.Rational(1e-198)
    approx_root_m = approx_root - sympy.Rational(1e-198)
    approx_roots.append(approx_root)
    is_sign_change = (
        gq_fn.subs(x, approx_root_p) > 0 and gq_fn.subs(x, approx_root_m) < 0
    ) or (gq_fn.subs(x, approx_root_p) < 0 and gq_fn.subs(x, approx_root_m) > 0)
    if is_sign_change:
      largest_sign_change = max(largest_sign_change, approx_root)

  return float(largest_sign_change)

Note that the score function uses the find_laguerre_combination(m, n, zs) method
to obtain the polynomial g(x) given z via exact fractional arithmetic in sympy.

You may code up any search method you want, and you are allowed to call the
get_score() function as many times as you want. You have access to it, you don't
need to code up the get_score() function.
You want the score it gives you to be as high as possible!

Your task is to write a search function that searches for the best list.
Your function will have 1000 seconds to run, and after that it has to have
returned the best construction it found. If after 1000 seconds it has not
returned anything, it will be terminated with negative infinity points. You can
use your time best if you have an outer loop of the form
"while time.time() - start_time < 1000:" or similar, just don't forget to define
the "start_time" variable early in your program.

In [ ]:
# @title Utils

import re
import sys
import time
from typing import Any, Callable, Mapping
import matplotlib.pyplot as plt
import numpy as np
import scipy.special
import sympy

sys.set_int_max_str_digits(40000)

In [ ]:
# @title Evaluation


def find_laguerre_combination(m, n, z):

  alpha = sympy.Rational(n, 2) - 1
  degrees = np.arange(1, 4 * m + 4, 2)
  x = sympy.symbols("x")
  lps = [
      sympy.polys.orthopolys.laguerre_poly(n=i, x=x, alpha=alpha, polys=False)
      for i in degrees
  ]

  num_lps = len(lps)
  num_conditions = 2 * m + 2  # Root at 0, double roots at z_i

  if num_lps < num_conditions:
    raise ValueError(
        "Not enough Laguerre polynomials to satisfy all conditions."
    )

  # Create a system of linear equations to solve for alpha_i
  A = sympy.Matrix(num_conditions, num_lps, lambda i, j: 0)
  b = sympy.Matrix(num_conditions, 1, lambda i, j: 0)

  b[1] = 1  # Set g'(0) = const

  # Condition 1: Root at 0 (g(0) = 0)
  for j in range(num_lps):
    A[0, j] = lps[j].subs(x, 0)
    A[1, j] = lps[j].diff(x).subs(x, 0)

  # Conditions 2 to 2m+2: Double roots at z_i (g(z_i) = 0 and g'(z_i) = 0)
  for i in range(0, m):
    zi = sympy.Rational(z[i])  # Ensure z_i is rational

    # g(z_i) = 0
    for j in range(num_lps):
      A[2 * i + 2, j] = lps[j].subs(x, zi)

    # g'(z_i) = 0 (derivative with respect to x)
    for j in range(num_lps):
      deriv_lp_j = lps[j].diff(x)  # Symbolic differentiation
      A[2 * i + 3, j] = deriv_lp_j.subs(x, zi)

  # Solve the linear system Ax = b for x (coefficients alpha_i)
  x = A.LUsolve(b)  # Use LU decomposition for efficient solving
  alpha_coeffs = [x[i] for i in range(num_lps)]
  g_fn = sum(alpha_coeffs[i] * lps[i] for i in range(len(alpha_coeffs)))
  return g_fn


ROOTS_NOT_ENFORCED = 10000
WRONG_Z_SHAPE = 10001
DERIVATIVES_NOT_ENFORCED = 10002
NO_SIGN_CHANGES = 10003
SUGGESTED_ROOTS_TOO_LARGE = 10004


def get_score(m: int, n: int, zs: np.ndarray | list[float]) -> float:
  """Returns the score of the given Laguerre combination."""
  if len(zs) != m:
    return WRONG_Z_SHAPE

  g_fn = find_laguerre_combination(m, n, zs)
  x = sympy.symbols('x')
  dg_fn = sympy.diff(g_fn, x)

  div = sympy.prod([(x - sympy.Rational(z)) ** 2 for z in zs]) * x
  gq_fn = sympy.exquo(g_fn, div)

  g_fn.subs(x, sympy.Rational(0))
  dg_fn.subs(x, sympy.Rational(1))

  for z in zs:
    if g_fn.subs(x, sympy.Rational(z)) != 0:
      return ROOTS_NOT_ENFORCED
    if dg_fn.subs(x, sympy.Rational(z)) != 0:
      return DERIVATIVES_NOT_ENFORCED

  real_roots = sympy.real_roots(gq_fn, x)
  if not real_roots:
    return NO_SIGN_CHANGES

  approx_roots = list()
  largest_sign_change = 0

  for root in real_roots:
    approx_root = root.eval_rational(n=200)
    approx_root_p = approx_root + sympy.Rational(1e-198)
    approx_root_m = approx_root - sympy.Rational(1e-198)
    approx_roots.append(approx_root)
    is_sign_change = (
        gq_fn.subs(x, approx_root_p) > 0 and gq_fn.subs(x, approx_root_m) < 0
    ) or (gq_fn.subs(x, approx_root_p) < 0 and gq_fn.subs(x, approx_root_m) > 0)
    if is_sign_change:
      largest_sign_change = max(largest_sign_change, approx_root)

  return float(largest_sign_change)

In [ ]:
# @title Initial program

suggested_roots = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])


def get_roots(m: int, n: int) -> list[float] | np.ndarray:
  """Generate roots to for optimal Laguerre combination."""
  variable_name = f'suggested_roots_{m}'
  best_roots = list(np.arange(1, m + 1) + n)

  if np.random.rand() < 0.5 and variable_name in globals():
    best_roots = globals()[variable_name]

  curr_positions = best_roots.copy()
  best_score = get_score(m, n, best_roots)

  start_time = time.time()
  while time.time() - start_time < 100:  # Search for 100 seconds
    random_index = np.random.randint(0, len(curr_positions))
    curr_positions[random_index:] += 1e-1 * np.random.randint(-10, 10)
    curr_score = get_score(n, curr_positions)
    if curr_score > best_score:
      best_score = curr_score
      best_roots = curr_positions.copy()
      print(f"Best score: {curr_score}")

  return best_roots

In [ ]:
# @title Configurations obtained by AlphaEvolve

# 6 roots in 25 dim
suggested_roots_6_25 = [38.59990403042335, 51.492514056486456, 64.61752892431971, 80.83626553375464, 99.10082276235605, 119.82181796581477]
# 8 roots in 25 dim
suggested_roots_8_25 = [38.52953040091796, 51.26018187250741, 63.71261940889087, 78.09254403415129, 93.6713541797075, 110.16959212745668, 132.72837654008953, 151.87758151262366]
# 10 roots in 25 dim
suggested_roots_10_25 = [38.446349008804845, 50.94894525741638, 64.42826066299327, 77.8810376282672, 91.22936339942667, 105.95375302939827, 117.75701066036017, 135.99677682063452, 154.36310605878, 179.31645167349984]
# 6 roots in 30 dim
suggested_roots_6_30 = [42.754060091336754, 55.93712498537983, 69.6672739058253, 85.74802171838502, 103.80696192096791, 122.6267287889253]
# 8 roots in 30 dim
suggested_roots_8_30 = [42.843076071461326, 53.59448994991431, 65.03632808656212, 79.0354644879144, 93.32647614615303, 108.29065789473482, 126.04751333162189, 137.2109969193287]
# 10 roots in 30 dim
suggested_roots_10_30 = [40.70608913194654, 54.367044004983676, 67.07359341470105, 83.95998264550133, 96.72724938921412, 107.46383275000353, 129.7690253337481, 142.0989363215454, 162.00144210663203, 177.46601777960245]
# 6 roots in 35 dim
suggested_roots_6_35 = [47.008355187949455, 60.459254120329135, 74.36584602868015, 91.6127685240848, 109.06949521970334, 129.01863659501757]
# 8 roots in 35 dim
suggested_roots_8_35 = [46.50466498749102, 60.715302405700946, 75.56008072687219, 88.35291756954845, 103.53056264712156, 125.59614192990736, 143.5176152969259, 160.14549435186555]
# 10 roots in 35 dim
suggested_roots_10_35 = [46.34839194715566, 59.086464707143136, 71.69493811810037, 85.28766727724832, 100.53582876782653, 114.11784957031244, 129.08209602859588, 150.99036977340384, 171.46491738755452, 200.0]

# Example scores
# get_score(10, 25, suggested_roots_10_25)
# get_score(10, 35, suggested_roots_10_35)

In [ ]:
# @title Examples of programs evolved by AlphaEvolve

def get_roots(m: int, n: int) -> list[float] | np.ndarray:
    """Generate roots to for optimal Laguerre combination."""
    rng = np.random.default_rng(seed=m * 100 + n)
    variable_name = f'suggested_roots_{m}'
    bounds = [(1, 200)] * m

    def root_fusion(roots, n, rng):
      """Combines current roots with random roots from a different range."""
      n_fusion = rng.integers(5, 5+ n//2)        #Scale it.

      # Scale for a better distribution in terms of range
      fused_roots = rng.uniform(1, n_fusion, size=len(roots))
      fusion_mask = rng.choice([True, False], size=len(roots), p=[0.21, 0.79]) #Mask

      roots[fusion_mask] = fused_roots[fusion_mask] #Mix now
      return roots

    def repulsive_init(num_points, lower_bound, upper_bound, n, m):
        """Generates a repulsive initialization within bounds."""
        points = np.linspace(lower_bound, upper_bound, num_points * 3) # more points.

        points = rng.choice(points, size=num_points, replace=False) # sample.

        points = np.sort(points)


        def force_repulsion(points, strength, iterations, n, m):
            points_arr = np.array(points)
            for _ in range(iterations): # iterate for better spread.
                for i in range(len(points_arr)):
                    force = 0.0
                    for j in range(len(points_arr)):
                        if i != j:
                            distance = points_arr[j] - points_arr[i]
                            # Cap the force to avoid extreme values
                            force += strength / max(abs(distance), 1e-6) * np.sign(distance)

                    points_arr[i] += 0.1*  force # Adaptive step
                points_arr = np.clip(points_arr, lower_bound, upper_bound)
                points_arr.sort() # Keep sorted
            return points_arr.tolist()

        strength = 0.5 * n + 0.1 * m  # Strength based on parameters
        return force_repulsion(points, strength, 50, n, m)   # A few iterations.

    def interpolate_roots(target_m: int, target_n: int, rng: np.random.Generator):
        """Interpolates roots from known solutions, with adaptive weighting."""
        known_solutions = {
            (11, 8): [37.705, 50.285, 62.893, 75.578, 88.454, 101.737, 115.776, 131.035, 148.162, 168.215, 193.766],
            # Add more known solutions if available
        }

        # Find the closest known (m, n)
        closest_mn = None
        min_distance = float('inf')
        for (km, kn), _ in known_solutions.items():
            distance = (km - target_m)**2 + (kn - target_n)**2
            if distance < min_distance:
                min_distance = distance
                closest_mn = (km, kn)

        #Dynamic Selection Of solutions and blend.
        num_sols_blend = min(3, len(known_solutions))
        dists = []
        sols = []
        for (km, kn), sol in known_solutions.items():   #Compute distance and sort
            dist = (km - target_m)**2 + (kn - target_n)**2
            dists.append(dist)
            sols.append(((km,kn),sol))

        closest_sols = sorted(zip(dists, sols), key = lambda x: x[0])[:num_sols_blend] #Take k closest

        if not closest_sols:
            return repulsive_init(target_m, 1, 200, target_n, target_m)


        #Blend by weightings.
        interpolated_roots = np.zeros(target_m) #Base values
        total_weight = 0.0
        for dist, sol_data in closest_sols:
            (known_m, known_n), known_roots = sol_data
            weight = np.exp(-dist / (target_m + target_n))
            total_weight+=weight

            interp_roots = np.interp(
                np.linspace(0, 1, target_m),  # Normalized indices for target_m
                np.linspace(0, 1, known_m),  # Normalized indices for known_m
                known_roots
            )
            interpolated_roots+= weight * interp_roots

        interpolated_roots /= total_weight
        known_m, known_n = closest_sols[0][1][0] #Get basic

        # Adaptive interpolation/extrapolation
        interpolated_roots = np.interp(
            np.linspace(0, 1, target_m),  # Normalized indices for target_m
            np.linspace(0, 1, known_m),  # Normalized indices for known_m
            known_roots
        ).tolist()

        # Adjust based on target n, with scaling
        for i in range(target_m):
            interpolated_roots[i] += (target_n - known_n) * (0.1 + 0.01 * target_m)  # dynamic scaling
            interpolated_roots[i] = np.clip(interpolated_roots[i] + rng.normal(0, (1 + target_n * 0.05) / (1 + target_m * 0.1)), 1, 200)

        return interpolated_roots

    # --- Initialization and Prior Knowledge ---
    use_prior = 0.5 + 0.4 * np.exp(-0.005 * m * n)
    jump_chance = 0.25 #MORE CHAOS. MORE RANDOM INIT.

    if rng.random() < jump_chance: # Rare Jump!
        #Less jumping!
        init_type = rng.choice(['repulsive',]) #No uniform (too much explosion)

        if init_type == 'repulsive':
            best_roots = repulsive_init(m, bounds[0][0], bounds[0][1], n, m) #Repulsive intit
        elif init_type == "root_fusion":
            best_roots = root_fusion(best_roots, n, rng)  # Call `initialize_roots` directly
        elif init_type == 'interpolated':
            best_roots = interpolate_roots(m, n, rng)
        elif init_type == 'cauchy':
            best_roots = rng.standard_cauchy(size=m) * (0.2 * n) + (n/2)  #Cauchy! Removing to reduce search space.

        else: # uniform
           best_roots = rng.uniform(bounds[0][0], bounds[0][1], size = m)
    else: # Regular prior or other inits
        if variable_name in globals() and rng.random() < use_prior:
            best_roots = np.array(globals()[variable_name]).copy()
            shrinkage = 0.9 + 0.01 * n
            best_roots = (shrinkage * best_roots) + rng.normal(0, (n ** 0.5) * 0.1, size=m)  # Smaller, scaled noise.
        else:
            if rng.random() < 0.7: # reduce chance it is interpolate
                best_roots = interpolate_roots(m, n, rng)  # Increased interpolation probability.
            else:
                # Fallback initialization.
                base_spacing = n / (1.5 + 0.05 * m)
                best_roots = np.array([i * (base_spacing + 0.017 * i * base_spacing) + n * (0.8 + 0.1 * rng.random()) for i in range(1, m + 1)])  # Adjusted.  Slightly more exponential to force local exploitation.

    best_roots = np.clip(best_roots, 1, 200)

    # --- Neighbor-Based Adjustments (Early Stage)---
    for _ in range(min(m // 2,3 )): # Reduced Neighbor adjust
        i = rng.integers(0, m)
        neighbors = []
        if i > 0:
            neighbors.append(best_roots[i - 1])
        if i < m - 1:
            neighbors.append(best_roots[i + 1])
        if neighbors:
             avg_neighbor = np.mean(neighbors)
             influence = 0.2 * (1 - m * 0.01)  # Reduce influence as m grows.
             best_roots[i] = (1 - influence) * best_roots[i] + influence * avg_neighbor
             best_roots[i] = np.clip(best_roots[i], 1, 200)

    # --- Gradient-Informed Perturbation (Core Optimization) ---
    if 'momentum' not in globals():
        globals()['momentum'] = np.zeros(m) # Init momentum

    # --- Hyper jump condition ---
    # Less often, MORE Hyperjumping/coordinate exploration
    num_gradient_steps = int(m * 0.7)  # Simplified num_gradient_steps

    #Reduced hyperspace jumps to 1 and 3, then add prime force.
    for _ in range(num_gradient_steps):
        i = rng.integers(0, m)

        local_best = best_roots.copy() # local back up
        if rng.random() < 0.01:  # Increased probability, try more often MORE DISTURBANCE, more jumping. More local back up

            #Prime fact
            prime_factor =  prime_factor_influence(best_roots, n)

            #Add small-scaling for exploration and perturbation strength control
            scale_rand= rng.uniform(-0.042 , 0.042) #Scale by a random number scaled, REMOVING, NO HYPER JUMP

            #Scale normalization: Adaptively scale based on n, with prime_influence
            scale = (np.mean(best_roots) + n * 1.51) / (np.linalg.norm(hyper_coords) + 1e-8) #Normalize hyperspace coordinates

            best_roots += np.array([scale * hyper_coords[k % HYPERDIM] for k in range(m)]) #Mix back to original values

            #Keep local, revert back
            delta_c = np.sum(best_roots - local_best)/m;
            if abs(delta_c) > .2 + rng.normal(0,.1111 ):
                best_roots = local_best # revere

        delta = 0.025 * n  # Smaller delta. Reduce it further from .2->.1 since removing HYPERSpace JUMP already jumps so far.

        # Central Difference Gradient Estimation
        original_root = best_roots[i]
        best_roots[i] = np.clip(original_root + delta, 1, 200)
        score_plus = get_score(m, n, best_roots)

        best_roots[i] = np.clip(original_root - delta, 1, 200)

        score_minus = get_score(m, n, best_roots)

        best_roots[i] = original_root  # Restore

        if score_plus == -float('inf') and score_minus == -float('inf'):         #Both bad.
            gradient = 0
        elif score_plus == -float('inf'):
            gradient = -1.0
        elif score_minus == -float('inf'):
            gradient = 1.0
        else:
            gradient = (score_plus - score_minus) / (2 * delta)

        #---- Newton Step
        # --- Random Gradient Scaling (Structured Exploration)---
        rand_scale = 1.0 + rng.normal(0, 0.325) # Small random scaling more aggressively scale down
        gradient *= rand_scale

        mu = 0.2 * n* rng.uniform(0.1, 0.9)   #Regularization parameter (randomness added here, and prime_influence)
        damping = 1 + mu
        descent_direction = gradient  / damping # Direction vector

        #----Momentum addition, helps navigate flat regions with randomness
        momentum_factor = 0.5 + rng.normal(0,0.05) #Randomized and bounded
        # --- Momentum Vector Perturbation ---
        momentum_noise = rng.normal(0, 0.02 * n)  #Small perturbation but bounded to small size
        globals()['momentum'][i] = momentum_factor * globals()['momentum'][i] + (1 - momentum_factor) * descent_direction + momentum_noise
        descent_direction = globals()['momentum'][i]


        max_step = 0.015 * n #limit each individual step.
        step_size = min(max_step/ abs(descent_direction)  + 1e-8, 0.05*n)  #Limit step taken
        # --- Step Size Randomization ---
        step_rand = 1.0 + rng.normal(0, 0.05)  # Randomize step AND ALSO factor number here. Reduced number factor since jumping causes problems
        best_roots[i] = np.clip(best_roots[i] - step_size * descent_direction * step_rand, 1, 200)


    #---Chaotic Burst Modification---
    if rng.random() < 0.038: # Probability of chaotic jump - Increased Chance to explore
        chaos = rng.normal(0, .25, m)  #Random values
        for j in range(m):
            best_roots[j] += n * 0.175 * chaos[j % m] #Apply series pertub.  Scale is also ramped.
            best_roots[j] = np.clip(best_roots[j], 1, 200) #Clip now.

    #-----LONG term global migration!-----
    #  High level re injection
    if 'long_contenders' not in globals():
        #Init set of long contender
        globals()['long_contenders'] = set()

    global_pull =.001/10 #What is global pool size?
    def _global():
        inject=[]
        if len(globals()['long_contenders']) > 3:
            cont_ = list(globals()['long_contenders'])
            c   = rng.choice(cont_)
            return c
        else:
             return ""

    global_item = _global()
    final_noise_scale = 0.009 * n + 0.009 * np.sqrt(np.arange(m)) #Reduced

    # global scale inject
    if global_item != "":
         print("Apply new random noise: global inject")
         new_noise     = rng.normal(0, final_noise_scale, size         = m)
         new_roots     = [min(200,max(1, new_noise[ind] +float(v))) for ind, v in enumerate(global_item)]
         return new_roots


    best_roots += rng.normal(0, final_noise_scale, size         = m)
    best_roots = np.clip(best_roots, 1, 200).tolist()


    return best_roots


def get_roots(m: int, n: int) -> list[float] | np.ndarray:
    """Generate roots to for optimal Laguerre combination."""
    rng = np.random.default_rng(seed=m * 100 + n)
    variable_name = f'suggested_roots_{m}'
    bounds = [(1, 200)] * m

    # --- Simulated Annealing Parameters ---
    # --- Simulated Annealing Parameters ---
    initial_temperature = 0.5 * (m + n)
    cooling_rate = 0.995
    pomodoro_interval = 20  # Define Pomodoro interval (cycles). To consider a good starting point for balancing exploration/exploitation
    cognitive_load = 0.08  # Define cognitive load (probability of distraction)
    random_perturbation_prob = 0.04 # Probability of strong random perturbation

    # --- State Management ---
    state = globals().get('state', { #Gets or reset
        'temperature': initial_temperature, #Reset to new initial vals
        'jump_scale': 0.1 * n,
        'pomodoro_counter': 0,  # Track Pomodoro intervals
        'best_score': float('-inf'),
        'solution_memory': deque(maxlen=5),
        'no_improvement_counter': 0,
        'best_roots': None,
    })
    temperature = state['temperature']
    jump_scale = state['jump_scale']
    best_score = state['best_score']
    solution_memory = state['solution_memory']
    no_improvement_counter = state['no_improvement_counter']
    pomodoro_counter = state['pomodoro_counter']

    if state['best_roots'] is None:  # First run, none provided.
        best_roots = np.array([rng.uniform(1, 200) for _ in range(m)])  #Initialize as well; we want to add historical ones.
        was_random_start = True    #Keep track.
    else:
        best_roots = state['best_roots'].copy()
        was_random_start = False #We didn't random start,



    # Initialization with Prior Knowledge
    use_prior = 0.3 + 0.3 * np.exp(-0.004 * m * n)
    if variable_name in globals() and rng.random() < use_prior:
        best_roots = np.array(globals()[variable_name]).copy()
        best_roots += rng.normal(0, temperature * 0.05 * n**0.3, size=m)   # Temperature dependent perturbation

    else:
        prime_nums = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97,
                      101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193,
                      197, 199]

        # Introduce a "Prime Resonance" factor + more dynamic factor to promote escape behavior during convergence
        prime_resonance = (prime_nums[m % len(prime_nums)] / prime_nums[-1]) * 0.02 + 0.005 * np.cos(time.time()*np.pi + m + n)  # small factor based on primes

        base_spacing = n / (2.0 + 0.04 * m) + 1.1 + jump_scale * np.sin(
            m * n / 5) + prime_resonance  # Jump scale and prime influence

        # Fibonacci-based perturbation
        fibonacci = [1, 1]
        for k in range(2, m):
            fibonacci.append((fibonacci[k-1] + fibonacci[k-2]) % 100) # Modulo to keep values small

        best_roots = np.array([
            i * base_spacing * (1 + 0.15 * (i / m)**1.5)  # Enhanced non-linear scaling (power law)
            * (1 + 0.01 * prime_nums[i % len(prime_nums)] / prime_nums[-1])  # Prime-based MODULATION of spacing
            * (1 + 0.005 * np.sin(i + prime_nums[i % len(prime_nums)]))  # Add Prime Resonance with sin
            + 0.95 * n + jump_scale * 0.1 * i * np.cos(i * m)  # Reduced jump scale influence
            + temperature * 0.01 * (fibonacci[i % m] / 100.0) * n  # Fibo.
            + (i & prime_nums[i % len(prime_nums)]) * 0.00009 * n #BitWise with Prime number
            * (1 + 0.001 * np.cos(np.sqrt(i+time.time()))) # More Modulation
            + 0.002 * n * np.sin(prime_nums[(i + int(time.time())) % len(prime_nums)]*i/(m+1))

            for i in range(1, m + 1)
        ])

    # --- Staccato Search (Pomodoro Technique) ---
    current_score = get_score(m, n, best_roots)  # Current score.
    best_score = globals().get('best_score', float('-inf'))

    # === Global State Management Reset ===
    reset_prob = (0.01 + pomodoro_counter*0.000025) #Adaptive
    memory_replay_prob = 0.45    # Try to replay past configs- can help recover from bad decisions,

    primes_used_for_reset = [3, 5, 7, 11, 13, 17]  # Primes to influence restarts
    prime_to_use = primes_used_for_reset[m % len(primes_used_for_reset)]

    # Check that best_roots is initialized
    is_initialized = variable_name in globals() or best_roots is not None # check.

    if is_initialized and pomodoro_counter > 0 and pomodoro_counter %prime_to_use == 0 and rng.random() < reset_prob:
        if len(state['solution_memory']) > 2 and rng.random() < memory_replay_prob:

            # Replay state history from the memory, make us eof other solutions.
            selected_solution_index = rng.choice(len(solution_memory))
            best_roots = solution_memory[selected_solution_index][0].copy() #Grab this old config.

            print(">>> REPLAY global best from state DICTIONARY <<<")
        else:
            # "Forget" the state by clearing the global dictionary.
            del globals()['state'] #DELETE THE STATE!
            state = {}
            temperature = initial_temperature
            jump_scale = 0.1 * n

            print(">>> STATE DICTIONARY RESET <<<")

    #Reduced mutation
    num_gradient_steps = int(m * 0.3)  # Further reduced gradient steps to let distractions work
    for _ in range(num_gradient_steps):
        i = rng.integers(0, m)
        delta = 0.02 * n  # Smaller delta for gradient estimation
        delta_2 = 0.008 * n  # Smaller delta for gradient estimation


        # Apply cognitive load/distraction. For robustness
        if rng.random() < cognitive_load * (1 + np.sin(time.time())):  # Temporal load variation (external/Zeitgeist stimulus).
             best_roots[i] += rng.normal(0, 0.08 * n)  # Small stochastic "annoyance."
             best_roots[i] = np.clip(best_roots[i], 1, 200)

        # Adaptive Step Size (and direction finding)

        original_root = best_roots[i]

        # Random Perturbation
        if rng.random() < random_perturbation_prob:
            best_roots[i] += rng.normal(0, n * 0.15) # Larger Perturbation
            best_roots[i] = np.clip(best_roots[i], 1, 200) # Clip and then evaluate.

        best_roots[i] = np.clip(original_root + delta, 1, 200)
        score_plus = get_score(m, n, best_roots)
        best_roots[i] = np.clip(original_root - delta, 1, 200)
        score_minus = get_score(m, n, best_roots)
        best_roots[i] = original_root  # Restore original root

        if score_plus == -float('inf') and score_minus == -float('inf'):
            gradient = 0.0
        elif score_plus == -float('inf'):
            gradient = -1.0
        elif score_minus == -float('inf'):
            gradient = 1.0
        else:
            gradient = (score_plus - score_minus) / (2 * delta)


        # Adaptive Step Size and Momentum (Smoother updates)
        # Higher learning rate if no recent improvements. Also inject number theoretic properties
        aggression_factor = 1.0 + 0.4 * np.tanh(no_improvement_counter / 20.0) * np.sin(prime_nums[i%len(prime_nums)])
        step_size = 0.03 * n * aggression_factor / (1 + np.abs(gradient))  # Adjust scaling here!

        momentum = 0.5  # Add momentum for smoother gradient descent

        # Ensure that momentum is initialized in the scope or increase!
        if 'momentum_values' not in locals():
            momentum_values = np.zeros(m)

        momentum_values[i] = momentum * momentum_values[i] + (1 - momentum) * step_size * gradient  # Momentum

        """
        A combinatorics augmented number theory-based perturbation is applied as part of the update, involving the injection of finely-tuned stochastic noise to enhance exploration within the solution space.
        """

        best_roots[i] = np.clip(best_roots[i] + momentum_values[i], 1, 200) # momentum step


    # --- Final Touches, reduced noise --- --Increase level - for final kick!
    noise = rng.normal(0, (n * 0.015 + np.sqrt(np.arange(m)) *0.05) * temperature * 0.08, size=m)  # Reduced final noise.

    best_roots = np.clip(best_roots + noise, 1, 200)  # reduced final noise.
    final_score = get_score(m, n, best_roots)

    # -- Stochastic Restart Mechanism --
    restart_prob = 0.001 + 0.005 * np.exp(no_improvement_counter / 40)  # Adaptive restart probability
    num_number_jump = len(prime_nums) # number list
    if rng.random() < restart_prob:        # More Restarts, bigger range and power laws.

        num_to_restart = rng.integers(1,min(m // 2 + 1, num_number_jump))  # Restart a portion of roots
        indices_to_restart = rng.choice(m, size=num_to_restart, replace=False)
        for j in indices_to_restart:    #Inject from primes now
            best_roots[j] = prime_nums[j % num_number_jump] + rng.normal(0,0.5 * n) # from prime number as seeding pt

        final_score = get_score(m, n, best_roots) #Update the scores

    current_shuffle_prob = 0.07 # Base probability
    current_shuffle_prob += (1 - np.exp(-0.001 * m * n)) * 0.5  # Adaptive, less if m*n large
    # --- "Aggressive Remix"- to force it out!---
    # This aggressively remixes the population.
    if rng.random() < current_shuffle_prob and best_roots is not None:  # If score gets bad.

        # Create radically different children.
        noise = rng.normal(0, 0.15 *n, size=m)
        prime_scaling = ( primes[m % len(primes)] * 0.01 / sum(primes[:min(m, len(primes))])) if m>0 else 0 # Small Prime Factors.

        recombined_roots = np.clip(best_roots *  np.sin(time.time()) + noise + prime_scaling, 1, 200) # Prime Mod
        best_roots =  recombined_roots

        final_score = get_score(m, n, best_roots)

    # -- Cache Update  --
    """
    "Breaks" are introduced periodically as cognitive interrupts. This method disrupts
    local search patterns to enhance long-term exploration, analogous to stepping away from a problem to gain fresh perspective.
    """

    # === "Break" Phase (Staccato Search) === # The interrupt part.

    break_probability = 0.11 #Aggressive breaks - highter chances
    break_type = rng.choice( ['root_reset', 'noise_injection'], p = [0.60, 0.40])  # Bias the selection towards root_reset:


    # Pomodoro is all or nothing
    if pomodoro_counter > 0 and pomodoro_counter %pomodoro_interval == 0 and rng.random() < break_probability:

        if break_type == 'root_reset':
            reset_indices = rng.choice(m, size=rng.integers(1, m // 3 + 1), replace=False)  # select random subset to be randomized.
            best_roots[reset_indices] = rng.uniform(1, 200, size=len(reset_indices))  #  assign random samples

        elif break_type == 'noise_injection':
            noise = rng.normal(0,0.25 * n, size=m) #Aggressive disturbance!
            best_roots = np.clip(best_roots + noise, 1, 200) # Add noise to the current best

    final_score = get_score(m, n, best_roots)
    # === End Break Phase -Pomodoro End! ===

    # ----- Update the globals  -----

    if final_score > best_score: #Update highest scores.

        best_score = final_score  #Store the final scores at the end

        #Overwrite (the best).
        state['best_score'] = best_score  # Store
        state['best_roots'] = best_roots.copy()  # Store current

        if variable_name in globals():
            globals()[variable_name] = best_roots.copy()# update as well
        else:
            globals()[variable_name] = best_roots.copy()
        no_improvement_counter = 0  # Reset on IMPROVEMENT

        # Solution memory implementation - track only improved sols
        is_present = any([(np.array_equal(best_roots, sol[0])) for sol in solution_memory])
        if memory_replay_prob>= 0.40 and not is_present: #Dont add duplicates (if larger )
            solution_memory.append((best_roots.copy(), best_score))# Keep at max 5, and score.


    jump_scale *= ( 1 + 0.013 * np.sin(pomodoro_counter + m + n))  #More dynamic
    jump_scale =  min(jump_scale, 0.518 * n)  # clip the jump scale

    # --Increase the no improve count--
    no_improvement_counter += 1

    #Add to Pomodoro counter- the more "staccato" this does, the larger it gets
    pomodoro_counter += 1   #Move first, important.


    """
    At conclusion of cycle, cache state is updated; the temperature parameter dictates annealing; jump scale enables variable disruption magnitude,
    whilst tracking both no_improvement_counter and pomodoro_counter contributes signal to phase transition decisions,
    that either exploit or abandon the incumbent best solution.
    """

    # Simplified State Cache - Only Store Best Score & Roots
    #Update State
    state['temperature'] = temperature
    state['jump_scale'] = jump_scale
    state['no_improvement_counter'] = no_improvement_counter  # State cache only stores the "count"!
    state['pomodoro_counter'] = pomodoro_counter  # Pomodoro counts.
    ##

    #If state resetted (we need to manually store) update
    if 'state' in locals():
        state['best_score'] = best_score
        state['solution_memory'] = solution_memory # solution memory

        state['best_roots'] = best_roots.copy() # Store back - this makes more sense now


        ##Finally, store the state, if its present.
        globals()['state'] = state #Global storage!

    return best_roots.tolist()


def get_roots(m: int, n: int) -> list[float] | np.ndarray:
    """Generate roots to for optimal Laguerre combination."""
    rng = np.random.default_rng(seed=m * 100 + n)
    variable_name = f'suggested_roots_{m}'
    bounds = [(1, 200)] * m

    # --- Simulated Annealing Parameters ---
    # --- Simulated Annealing Parameters ---
    initial_temperature = 0.5 * (m + n)
    cooling_rate = 0.995
    pomodoro_interval = 20  # Define Pomodoro interval (cycles). To consider a good starting point for balancing exploration/exploitation
    cognitive_load = 0.08  # Define cognitive load (probability of distraction)
    random_perturbation_prob = 0.04 # Probability of strong random perturbation

    # --- State Management ---
    state = globals().get('state', { #Gets or reset
        'temperature': initial_temperature, #Reset to new initial vals
        'jump_scale': 0.1 * n,
        'pomodoro_counter': 0,  # Track Pomodoro intervals
        'best_score': float('-inf'),
        'solution_memory': deque(maxlen=5),
        'no_improvement_counter': 0,
        'best_roots': None,
    })
    temperature = state['temperature']
    jump_scale = state['jump_scale']
    best_score = state['best_score']
    solution_memory = state['solution_memory']
    no_improvement_counter = state['no_improvement_counter']
    pomodoro_counter = state['pomodoro_counter']

    if state['best_roots'] is None:  # First run, none provided.
        best_roots = np.array([rng.uniform(1, 200) for _ in range(m)])  #Initialize as well; we want to add historical ones.
        was_random_start = True    #Keep track.
    else:
        best_roots = state['best_roots'].copy()
        was_random_start = False #We didn't random start,



    # Initialization with Prior Knowledge
    use_prior = 0.3 + 0.3 * np.exp(-0.004 * m * n)
    if variable_name in globals() and rng.random() < use_prior:
        best_roots = np.array(globals()[variable_name]).copy()
        best_roots += rng.normal(0, temperature * 0.05 * n**0.3, size=m)   # Temperature dependent perturbation

    else:
        prime_nums = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97,
                      101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193,
                      197, 199]

        # Introduce a "Prime Resonance" factor + more dynamic factor to promote escape behavior during convergence
        prime_resonance = (prime_nums[m % len(prime_nums)] / prime_nums[-1]) * 0.02 + 0.005 * np.cos(time.time()*np.pi + m + n)  # small factor based on primes

        base_spacing = n / (2.0 + 0.04 * m) + 1.1 + jump_scale * np.sin(
            m * n / 5) + prime_resonance  # Jump scale and prime influence

        # Fibonacci-based perturbation
        fibonacci = [1, 1]
        for k in range(2, m):
            fibonacci.append((fibonacci[k-1] + fibonacci[k-2]) % 100) # Modulo to keep values small

        best_roots = np.array([
            i * base_spacing * (1 + 0.15 * (i / m)**1.5)  # Enhanced non-linear scaling (power law)
            * (1 + 0.01 * prime_nums[i % len(prime_nums)] / prime_nums[-1])  # Prime-based MODULATION of spacing
            * (1 + 0.005 * np.sin(i + prime_nums[i % len(prime_nums)]))  # Add Prime Resonance with sin
            + 0.95 * n + jump_scale * 0.1 * i * np.cos(i * m)  # Reduced jump scale influence
            + temperature * 0.01 * (fibonacci[i % m] / 100.0) * n  # Fibo.
            + (i & prime_nums[i % len(prime_nums)]) * 0.00009 * n #BitWise with Prime number
            * (1 + 0.001 * np.cos(np.sqrt(i+time.time()))) # More Modulation
            + 0.002 * n * np.sin(prime_nums[(i + int(time.time())) % len(prime_nums)]*i/(m+1))

            for i in range(1, m + 1)
        ])

    # --- Staccato Search (Pomodoro Technique) ---
    current_score = get_score(m, n, best_roots)  # Current score.
    best_score = globals().get('best_score', float('-inf'))

    # === Global State Management Reset ===
    reset_prob = (0.01 + pomodoro_counter*0.000025) #Adaptive
    memory_replay_prob = 0.45    # Try to replay past configs- can help recover from bad decisions,

    primes_used_for_reset = [3, 5, 7, 11, 13, 17]  # Primes to influence restarts
    prime_to_use = primes_used_for_reset[m % len(primes_used_for_reset)]

    # Check that best_roots is initialized
    is_initialized = variable_name in globals() or best_roots is not None # check.

    if is_initialized and pomodoro_counter > 0 and pomodoro_counter %prime_to_use == 0 and rng.random() < reset_prob:
        if len(state['solution_memory']) > 2 and rng.random() < memory_replay_prob:

            # Replay state history from the memory, make us eof other solutions.
            selected_solution_index = rng.choice(len(solution_memory))
            best_roots = solution_memory[selected_solution_index][0].copy() #Grab this old config.

            print(">>> REPLAY global best from state DICTIONARY <<<")
        else:
            # "Forget" the state by clearing the global dictionary.
            del globals()['state'] #DELETE THE STATE!
            state = {}
            temperature = initial_temperature
            jump_scale = 0.1 * n

            print(">>> STATE DICTIONARY RESET <<<")

    #Reduced mutation
    num_gradient_steps = int(m * 0.3)  # Further reduced gradient steps to let distractions work
    for _ in range(num_gradient_steps):
        i = rng.integers(0, m)
        delta = 0.02 * n  # Smaller delta for gradient estimation
        delta_2 = 0.008 * n  # Smaller delta for gradient estimation


        # Apply cognitive load/distraction. For robustness
        if rng.random() < cognitive_load * (1 + np.sin(time.time())):  # Temporal load variation (external/Zeitgeist stimulus).
             best_roots[i] += rng.normal(0, 0.08 * n)  # Small stochastic "annoyance."
             best_roots[i] = np.clip(best_roots[i], 1, 200)

        # Adaptive Step Size (and direction finding)

        original_root = best_roots[i]

        # Random Perturbation
        if rng.random() < random_perturbation_prob:
            best_roots[i] += rng.normal(0, n * 0.15) # Larger Perturbation
            best_roots[i] = np.clip(best_roots[i], 1, 200) # Clip and then evaluate.

        best_roots[i] = np.clip(original_root + delta, 1, 200)
        score_plus = get_score(m, n, best_roots)
        best_roots[i] = np.clip(original_root - delta, 1, 200)
        score_minus = get_score(m, n, best_roots)
        best_roots[i] = original_root  # Restore original root

        if score_plus == -float('inf') and score_minus == -float('inf'):
            gradient = 0.0
        elif score_plus == -float('inf'):
            gradient = -1.0
        elif score_minus == -float('inf'):
            gradient = 1.0
        else:
            gradient = (score_plus - score_minus) / (2 * delta)


        # Adaptive Step Size and Momentum (Smoother updates)
        # Higher learning rate if no recent improvements. Also inject number theoretic properties
        aggression_factor = 1.0 + 0.4 * np.tanh(no_improvement_counter / 20.0) * np.sin(prime_nums[i%len(prime_nums)])
        step_size = 0.03 * n * aggression_factor / (1 + np.abs(gradient))  # Adjust scaling here!

        momentum = 0.5  # Add momentum for smoother gradient descent

        # Ensure that momentum is initialized in the scope or increase!
        if 'momentum_values' not in locals():
            momentum_values = np.zeros(m)

        momentum_values[i] = momentum * momentum_values[i] + (1 - momentum) * step_size * gradient  # Momentum

        """
        A combinatorics augmented number theory-based perturbation is applied as part of the update, involving the injection of finely-tuned stochastic noise to enhance exploration within the solution space.
        """

        best_roots[i] = np.clip(best_roots[i] + momentum_values[i], 1, 200) # momentum step


    # --- Final Touches, reduced noise --- --Increase level - for final kick!
    noise = rng.normal(0, (n * 0.015 + np.sqrt(np.arange(m)) *0.05) * temperature * 0.08, size=m)  # Reduced final noise.

    best_roots = np.clip(best_roots + noise, 1, 200)  # reduced final noise.
    final_score = get_score(m, n, best_roots)

    # -- Stochastic Restart Mechanism --
    restart_prob = 0.001 + 0.005 * np.exp(no_improvement_counter / 40)  # Adaptive restart probability
    num_number_jump = len(prime_nums) # number list
    if rng.random() < restart_prob:        # More Restarts, bigger range and power laws.

        num_to_restart = rng.integers(1,min(m // 2 + 1, num_number_jump))  # Restart a portion of roots
        indices_to_restart = rng.choice(m, size=num_to_restart, replace=False)
        for j in indices_to_restart:    #Inject from primes now
            best_roots[j] = prime_nums[j % num_number_jump] + rng.normal(0,0.5 * n) # from prime number as seeding pt

        final_score = get_score(m, n, best_roots) #Update the scores

    current_shuffle_prob = 0.07 # Base probability
    current_shuffle_prob += (1 - np.exp(-0.001 * m * n)) * 0.5  # Adaptive, less if m*n large
    # --- "Aggressive Remix"- to force it out!---
    # This aggressively remixes the population.
    if rng.random() < current_shuffle_prob and best_roots is not None:  # If score gets bad.

        # Create radically different children.
        noise = rng.normal(0, 0.15 *n, size=m)
        prime_scaling = ( primes[m % len(primes)] * 0.01 / sum(primes[:min(m, len(primes))])) if m>0 else 0 # Small Prime Factors.

        recombined_roots = np.clip(best_roots *  np.sin(time.time()) + noise + prime_scaling, 1, 200) # Prime Mod
        best_roots =  recombined_roots

        final_score = get_score(m, n, best_roots)

    # -- Cache Update  --
    """
    "Breaks" are introduced periodically as cognitive interrupts. This method disrupts
    local search patterns to enhance long-term exploration, analogous to stepping away from a problem to gain fresh perspective.
    """

    # === "Break" Phase (Staccato Search) === # The interrupt part.

    break_probability = 0.11 #Aggressive breaks - highter chances
    break_type = rng.choice( ['root_reset', 'noise_injection'], p = [0.60, 0.40])  # Bias the selection towards root_reset:


    # Pomodoro is all or nothing
    if pomodoro_counter > 0 and pomodoro_counter %pomodoro_interval == 0 and rng.random() < break_probability:

        if break_type == 'root_reset':
            reset_indices = rng.choice(m, size=rng.integers(1, m // 3 + 1), replace=False)  # select random subset to be randomized.
            best_roots[reset_indices] = rng.uniform(1, 200, size=len(reset_indices))  #  assign random samples

        elif break_type == 'noise_injection':
            noise = rng.normal(0,0.25 * n, size=m) #Aggressive disturbance!
            best_roots = np.clip(best_roots + noise, 1, 200) # Add noise to the current best

    final_score = get_score(m, n, best_roots)
    # === End Break Phase -Pomodoro End! ===

    # ----- Update the globals  -----

    if final_score > best_score: #Update highest scores.

        best_score = final_score  #Store the final scores at the end

        #Overwrite (the best).
        state['best_score'] = best_score  # Store
        state['best_roots'] = best_roots.copy()  # Store current

        if variable_name in globals():
            globals()[variable_name] = best_roots.copy()# update as well
        else:
            globals()[variable_name] = best_roots.copy()
        no_improvement_counter = 0  # Reset on IMPROVEMENT

        # Solution memory implementation - track only improved sols
        is_present = any([(np.array_equal(best_roots, sol[0])) for sol in solution_memory])
        if memory_replay_prob>= 0.40 and not is_present: #Dont add duplicates (if larger )
            solution_memory.append((best_roots.copy(), best_score))# Keep at max 5, and score.


    jump_scale *= ( 1 + 0.013 * np.sin(pomodoro_counter + m + n))  #More dynamic
    jump_scale =  min(jump_scale, 0.518 * n)  # clip the jump scale

    # --Increase the no improve count--
    no_improvement_counter += 1

    #Add to Pomodoro counter- the more "staccato" this does, the larger it gets
    pomodoro_counter += 1   #Move first, important.


    """
    At conclusion of cycle, cache state is updated; the temperature parameter dictates annealing; jump scale enables variable disruption magnitude,
    whilst tracking both no_improvement_counter and pomodoro_counter contributes signal to phase transition decisions,
    that either exploit or abandon the incumbent best solution.
    """

    # Simplified State Cache - Only Store Best Score & Roots
    #Update State
    state['temperature'] = temperature
    state['jump_scale'] = jump_scale
    state['no_improvement_counter'] = no_improvement_counter  # State cache only stores the "count"!
    state['pomodoro_counter'] = pomodoro_counter  # Pomodoro counts.
    ##

    #If state resetted (we need to manually store) update
    if 'state' in locals():
        state['best_score'] = best_score
        state['solution_memory'] = solution_memory # solution memory

        state['best_roots'] = best_roots.copy() # Store back - this makes more sense now


        ##Finally, store the state, if its present.
        globals()['state'] = state #Global storage!

    return best_roots.tolist()


## Uncertainty principles

**Prompt for the search setup**

You are a research mathematician and an expert software developer.
Your task is to optimize linear combinations of Laguerre polynomials with
certain properties.

For a given pair of natural number m we consider the following
construction.

Let $g_k(x) = L_k^\alpha(x)$ where $L_k^\alpha(x)$ denotes the standard
generalized Laguerre polynomial of degree k and order $\alpha = (1 / 2) - 1$.

First, select a collection of m positive real numbers z_1, z_2, \dots, z_m.

Then let g(x) be a linear combination of even degree g_k's up to order $4m + 3$
i.e. a linear sum of g_0, g_2, \dots, g_(4m + 2). The coefficients of this
linear combination are chosen such that z_1, \dots, z_m are double roots and
g(0) = 0 and g'(0) = 1 (Counting degrees of freedom implies that there is a
unique such linear combination g(x)).

Finally, define $r$ to be the largest positive sign change of the polynomial
g(x).

GOAL:
Your optimization task is, for a given natural number m, to find a
collection of m positive real numbers z_1, z_2, \dots, z_m such that the
polynomial g(x) has the smallest possible value of r.

Specifically, the Python function you have to provide has the following
signature:

def get_roots(m: int) -> list[float] | np.ndarray:

The function get_roots returns the one-dimensional array z of size m consisting
of the positive real numbers $z_i$ for $i = 1, \dots, m$.

HINTS:
1. For m = 5, the following list is a good initial guess:
[3.6331003, 5.6714292, 33.09981679, 41.1543366, 50.98385922]

2. Try to find constructions of roots that are not too spread apart or too
large. The absolute values of the roots z_i should be not larger than 300.

The score of z is given by the corresponding value of r. To compute this use
precise fractional arithmetic in sympy.

The exact score function your construction will be evaluated on is given below:

def get_score(zs: np.ndarray | list[float]) -> float:
  """Returns the score of the given Laguerre combination."""

  g_fn = find_laguerre_combination(zs)
  x = sympy.symbols('x')
  dg_fn = sympy.diff(g_fn, x)

  div = sympy.prod([(x - sympy.Rational(z)) ** 2 for z in zs]) * x
  gq_fn = sympy.exquo(g_fn, div)

  g_fn.subs(x, sympy.Rational(0))
  dg_fn.subs(x, sympy.Rational(1))

  for z in zs:
    if g_fn.subs(x, sympy.Rational(z)) != 0:
      return ROOTS_NOT_ENFORCED
    if dg_fn.subs(x, sympy.Rational(z)) != 0:
      return DERIVATIVES_NOT_ENFORCED

  real_roots = sympy.real_roots(gq_fn, x)
  if not real_roots:
    return NO_SIGN_CHANGES

  approx_roots = list()
  largest_sign_change = 0

  for root in real_roots:
    approx_root = root.eval_rational(n=200)
    approx_root_p = approx_root + sympy.Rational(1e-198)
    approx_root_m = approx_root - sympy.Rational(1e-198)
    approx_roots.append(approx_root)
    is_sign_change = (
        gq_fn.subs(x, approx_root_p) > 0 and gq_fn.subs(x, approx_root_m) < 0
    ) or (gq_fn.subs(x, approx_root_p) < 0 and gq_fn.subs(x, approx_root_m) > 0)
    if is_sign_change:
      largest_sign_change = max(largest_sign_change, approx_root)

  return np.sqrt(float(largest_sign_change) / 2 / np.pi)

Note that the score function uses the find_laguerre_combination(zs) method
to obtain the polynomial g(x) given z via exact fractional arithmetic in sympy.

You may code up any search method you want, and you are allowed to call the
get_score() function as many times as you want. You have access to it, you don't
need to code up the get_score() function.
You want the score it gives you to be as high as possible!

Your task is to write a search function that searches for the best list.
Your function will have 1000 seconds to run, and after that it has to have
returned the best construction it found. If after 1000 seconds it has not
returned anything, it will be terminated with negative infinity points. You can
use your time best if you have an outer loop of the form
"while time.time() - start_time < 1000:" or similar, just don't forget to define
the "start_time" variable early in your program. You may choose n, the length of
the list, to be anything in this experiment. Larger values of n have more
potential for bigger scores, but the search space is larger so they are harder
to find. You'll have to balance it accordingly!

**Prompts for search setup with starting hints for 5 roots**

You are a research mathematician and an expert software developer.
Your task is to optimize linear combinations of Laguerre polynomials with
certain properties.

For a given pair of natural number m we consider the following
construction.

Let $g_k(x) = L_k^\alpha(x)$ where $L_k^\alpha(x)$ denotes the standard
generalized Laguerre polynomial of degree k and order $\alpha = (1 / 2) - 1$.

First, select a collection of m positive real numbers z_1, z_2, \dots, z_m.

Then let g(x) be a linear combination of even degree g_k's up to order $4m + 3$
i.e. a linear sum of g_0, g_2, \dots, g_(4m + 2). The coefficients of this
linear combination are chosen such that z_1, \dots, z_m are double roots and
g(0) = 0 and g'(0) = 1 (Counting degrees of freedom implies that there is a
unique such linear combination g(x)).

Finally, define $r$ to be the largest positive sign change of the polynomial
g(x).

GOAL:
Your optimization task is, for a given natural number m, to find a
collection of m positive real numbers z_1, z_2, \dots, z_m such that the
polynomial g(x) has the smallest possible value of r.

Specifically, the Python function you have to provide has the following
signature:

def get_roots(m: int) -> list[float] | np.ndarray:

The function get_roots returns the one-dimensional array z of size m consisting
of the positive real numbers $z_i$ for $i = 1, \dots, m$.

HINTS:
1. For m = 5, the following list is a good initial guess:
[3.6331003, 5.6714292, 33.09981679, 41.1543366, 50.98385922]
Similarly, for m = 2 a good initial guess is:
[5.00115031, 42.3186769]
For m = 6, the following list is a good initial guess:
[41.65007642, 40.31923115, 40.14619073, 5.54073594, 45.31997179, 3.60084242]
2. For m = 6 try using the above construction for m = 5 and adding a new root
e.g. around 60. Then try optimizing the roots further starting from there.
3. Try to find constructions of roots that are not too spread apart or too
large. The absolute values of the roots z_i should be not larger than 300.

The score of z is given by the corresponding value of r. To compute this use
precise fractional arithmetic in sympy.

SCORING:
The exact score function your construction will be evaluated on is given below:

def get_score(zs: np.ndarray | list[float]) -> float:
  """Returns the score of the given Laguerre combination."""

  g_fn = find_laguerre_combination(zs)
  x = sympy.symbols('x')
  dg_fn = sympy.diff(g_fn, x)

  div = sympy.prod([(x - sympy.Rational(z)) ** 2 for z in zs]) * x
  gq_fn = sympy.exquo(g_fn, div)

  g_fn.subs(x, sympy.Rational(0))
  dg_fn.subs(x, sympy.Rational(1))

  for z in zs:
    if g_fn.subs(x, sympy.Rational(z)) != 0:
      return -ROOTS_NOT_ENFORCED
    if dg_fn.subs(x, sympy.Rational(z)) != 0:
      return -DERIVATIVES_NOT_ENFORCED

  real_roots = sympy.real_roots(gq_fn, x)
  if not real_roots:
    return -NO_SIGN_CHANGES

  approx_roots = list()
  largest_sign_change = 0

  for root in real_roots:
    approx_root = root.eval_rational(n=200)
    approx_root_p = approx_root + sympy.Rational(1e-198)
    approx_root_m = approx_root - sympy.Rational(1e-198)
    approx_roots.append(approx_root)
    is_sign_change = (
        gq_fn.subs(x, approx_root_p) > 0 and gq_fn.subs(x, approx_root_m) < 0
    ) or (gq_fn.subs(x, approx_root_p) < 0 and gq_fn.subs(x, approx_root_m) > 0)
    if is_sign_change:
      largest_sign_change = max(largest_sign_change, approx_root)

  return -np.sqrt(float(largest_sign_change) / 2 / np.pi)

Note that the score function uses the find_laguerre_combination(zs) method
to obtain the polynomial g(x) given z via exact fractional arithmetic in sympy.

You may code up any search method you want, and you are allowed to call the
get_score() function as many times as you want. You have access to it, you don't
need to code up the get_score() function.
You want the score it gives you to be as high as possible!

Your task is to write a search function that searches for the best list.
Your function will have 1000 seconds to run, and after that it has to have
returned the best construction it found. If after 1000 seconds it has not
returned anything, it will be terminated with negative infinity points. You can
use your time best if you have an outer loop of the form
"while time.time() - start_time < 1000:" or similar, just don't forget to define
the "start_time" variable early in your program. You may choose m, the length of
the list, to be anything in this experiment. Larger values of m have more
potential for bigger scores, but the search space is larger so they are harder
to find. You'll have to balance it accordingly!

In [ ]:
# @title Evaluation

ROOTS_NOT_ENFORCED = 10000
WRONG_Z_SHAPE = 10001
DERIVATIVES_NOT_ENFORCED = 10002
NO_SIGN_CHANGES = 10003
SUGGESTED_ROOTS_TOO_LARGE = 10004


def find_laguerre_combination(z) -> sympy.Expr:
  """Computing Laguerre combinations for given roots."""
  n = 1
  m = len(z)
  alpha = sympy.Rational(n, 2) - 1
  degrees = np.arange(0, 4 * m + 4, 2)
  x = sympy.symbols('x')
  lps = [
      sympy.polys.orthopolys.laguerre_poly(n=i, x=x, alpha=alpha, polys=False)
      for i in degrees
  ]

  num_lps = len(lps)
  num_conditions = 2 * m + 2  # Root at 0, double roots at z_i

  if num_lps < num_conditions:
    raise ValueError(
        'Not enough Laguerre polynomials to satisfy all conditions.'
    )

  # Create a system of linear equations to solve for alpha_i
  mat = sympy.Matrix(num_conditions, num_lps, lambda i, j: 0)
  b = sympy.Matrix(num_conditions, 1, lambda i, j: 0)

  b[1] = 1  # Set g'(0) = const

  # Condition 1: Root at 0 (g(0) = 0)
  for j in range(num_lps):
    mat[0, j] = lps[j].subs(x, 0)
    mat[1, j] = lps[j].diff(x).subs(x, 0)

  # Conditions 2 to 2m+2: Double roots at z_i (g(z_i) = 0 and g'(z_i) = 0)
  for i in range(0, m):
    zi = sympy.Rational(z[i])

    # g(z_i) = 0
    for j in range(num_lps):
      mat[2 * i + 2, j] = lps[j].subs(x, zi)

    # g'(z_i) = 0 (derivative with respect to x)
    for j in range(num_lps):
      deriv_lp_j = lps[j].diff(x)
      mat[2 * i + 3, j] = deriv_lp_j.subs(x, zi)

  # Solve the linear system Ax = b for x (coefficients alpha_i)
  x = mat.LUsolve(b)
  alpha_coeffs = [x[i] for i in range(num_lps)]
  g_fn = sum(alpha_coeffs[i] * lps[i] for i in range(len(alpha_coeffs)))
  return g_fn


def get_score(zs: np.ndarray | list[float]) -> float:
  """Returns the score of the given Laguerre combination."""

  g_fn = find_laguerre_combination(zs)
  x = sympy.symbols('x')
  dg_fn = sympy.diff(g_fn, x)

  div = sympy.prod([(x - sympy.Rational(z)) ** 2 for z in zs]) * x
  gq_fn = sympy.exquo(g_fn, div)

  g_fn.subs(x, sympy.Rational(0))
  dg_fn.subs(x, sympy.Rational(1))

  for z in zs:
    if g_fn.subs(x, sympy.Rational(z)) != 0:
      return -ROOTS_NOT_ENFORCED
    if dg_fn.subs(x, sympy.Rational(z)) != 0:
      return -DERIVATIVES_NOT_ENFORCED

  real_roots = sympy.real_roots(gq_fn, x)
  if not real_roots:
    return -NO_SIGN_CHANGES

  approx_roots = list()
  largest_sign_change = 0

  for root in real_roots:
    approx_root = root.eval_rational(n=200)
    approx_root_p = approx_root + sympy.Rational(1e-198)
    approx_root_m = approx_root - sympy.Rational(1e-198)
    approx_roots.append(approx_root)
    is_sign_change = (
        gq_fn.subs(x, approx_root_p) > 0 and gq_fn.subs(x, approx_root_m) < 0
    ) or (gq_fn.subs(x, approx_root_p) < 0 and gq_fn.subs(x, approx_root_m) > 0)
    if is_sign_change:
      largest_sign_change = max(largest_sign_change, approx_root)

  return -np.sqrt(float(largest_sign_change) / 2 / np.pi)



In [ ]:
# @title Initial programs


def get_roots(m: int) -> list[float] | np.ndarray:
  """Generate roots to for optimal Laguerre combination."""
  variable_name = f'suggested_roots_{m}'

  best_roots = np.arange(1, m + 1)

  if m == 2:
    best_roots = [5.00115031, 42.3186769]

  if np.random.rand() < 0.5 and variable_name in globals():
    best_roots = globals()[variable_name]

  curr_positions = best_roots.copy()
  best_score = get_score(best_roots)

  start_time = time.time()
  while time.time() - start_time < 100:  # Search for 100 seconds
    random_index = np.random.randint(0, len(curr_positions))
    curr_positions[random_index:] += 1e-1 * np.random.randint(-10, 10)
    curr_score = get_score(curr_positions)
    if curr_score > best_score:
      best_score = curr_score
      best_roots = curr_positions.copy()
      print(f"Best score: {best_score}")

  return best_roots

In [ ]:
# @title Constructions obtained by AlphaEvolve

# 6 roots
z_6 = [3.64273649, 5.68246114, 33.00463486, 40.97185579, 50.1028231, 53.76768016]
# 7 roots
z_7 = [3.64913287, 5.67235784, 38.79096469, 32.62677356, 45.48028355, 52.97276933, 106.77886152]
# 8 roots
z_8 = [3.64386938, 5.69329786, 32.38322129, 38.90891377, 45.14892756, 53.11575866, 99.06784500, 122.102121266]
# 9 roots
z_9 = [3.65229523, 5.69674475, 32.13629449, 38.30580848, 44.53027128, 52.78630070, 98.67722817, 118.22167413, 133.59986194]
# 10 roots
z_10 = [3.6331003, 5.6714292, 33.09981679, 38.35917516, 41.1543366, 50.98385922, 59.75317169, 94.27439607, 119.86075361, 136.35793559]


# Upper bounds on the uncertainty constant e.g.:
# get_score(z_7)**2
# get_score(z_8)**2

In [ ]:
# @title Programs obtained by AlphaEvolve



def get_roots(m: int) -> list[float] | np.ndarray:
  """Generate roots to for optimal Laguerre combination."""

  # Set a seed for reproducibility (Optional, removed for more varied runs)
  # np.random.seed(42)

  # Start optimization time
  start_time = time.time()

  # Flag to indicate if Nelder-Mead has made recent progress.  Reset every iteration.
  nelder_mead_progress = False

  best_score = -float('inf')
  best_roots = None
  bounds = [(1e-9, 300.0)] * m

  # Initialize global best roots list
  global_best_roots_list = []

  # Start optimization time
  start_time = time.time()

  # Pre-defined good initial guesses
  known_good_roots = {
      1: np.array([40.0]),
      2: np.array([5.00115031, 42.3186769]),
      3: np.array([4.0, 5.0, 42.0]), # Example: simple structured guess
      4: np.array([4.0, 5.0, 40.0, 45.0]), # Example: simple structured guess
      5: np.array([3.6331003, 5.6714292, 33.09981679, 41.1543366, 50.98385922]),
      6: np.array([3.60084242, 5.54073594, 40.14619073, 40.31923115, 41.65007642, 45.31997179]), # Sorted version of the hint
      7: np.array([3.5, 5.5, 30.0, 35.0, 40.0, 45.0, 50.0]), # Structured guess for m=7
      8: np.array([3.5, 5.5, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0]), # Structured guess for m=8
      12: np.array([3.5, 5.5, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0]) # Structured guess for m=12
  }

  initial_guesses_list = []

  # Strategy 1: Use known good guesses
  if m in known_good_roots:
      initial_guesses_list.append(np.sort(known_good_roots[m]))

  # Strategy 2: Structured guesses for other m
  if m > 6:
      # Approach 2a: Two small roots, rest spaced out (refined)
      num_small = 2
      if m - num_small >= 0:
         small_roots = [4.0, 5.0]
         if m > num_small:
             start_val_a = 30.0 + (m-num_small) * 2.5
             end_val_a = 70.0 + (m-num_small) * 3 + m
             large_roots_a = list(np.linspace(start_val_a, end_val_a, m - num_small))
             initial_guesses_list.append(np.sort(np.array(small_roots + large_roots_a)))
         elif m == num_small:
              initial_guesses_list.append(np.sort(np.array(small_roots)))

      # Add another structured guess: spaced out with increasing gaps (exponential)
      if m > 1:
          structured_start_exp = 5.0
          structured_end_exp = 150.0 + m * 8 # Increased end range
          # Use log space for a more exponential-like spacing
          structured_roots_exp = np.exp(np.linspace(np.log(structured_start_exp), np.log(structured_end_exp), m))
          initial_guesses_list.append(np.sort(np.clip(np.array(structured_roots_exp), bounds[0][0], bounds[0][1])))

      # Add a structured guess with roots clustered at the lower end and then spaced out
      if m > 2:
          num_clustered = max(2, int(m * 0.2)) # Cluster a percentage of roots
          clustered_roots = np.linspace(4.0, 10.0 + m*0.5, num_clustered)
          if m > num_clustered:
              remaining_roots_start = clustered_roots[-1] + 10.0
              remaining_roots_end = 200.0 + m * 10
              remaining_roots = np.linspace(remaining_roots_start, remaining_roots_end, m - num_clustered)
              initial_guesses_list.append(np.sort(np.clip(np.concatenate([clustered_roots, remaining_roots]), bounds[0][0], bounds[0][1])))


  # Strategy 3: Add some random guesses
  num_random_guesses = 4 if m <= 10 else 2 # Keep a few random for diversity
  for _ in range(num_random_guesses):
       random_guess = np.random.uniform(1.0, 200.0 + m*10, m) # Range scaled with m
       initial_guesses_list.append(np.sort(np.array(random_guess)))

  # Add perturbations of known good roots for smaller m (sorted)
  if m > 2:
      for prev_m in range(2, min(m, 13)): # Extended check for known small m values
          if prev_m in known_good_roots:
              prev_roots = known_good_roots[prev_m]
              if m - prev_m > 0:
                  for _ in range(3): # Add more permutations with new roots
                      # Add new roots in a reasonable range
                      new_roots_added = np.random.uniform(30.0, 150.0 + m*5, m - prev_m)
                      combined_roots = np.concatenate([prev_roots, new_roots_added])
                      if len(combined_roots) == m:
                           initial_guesses_list.append(np.sort(np.clip(combined_roots, bounds[0][0], bounds[0][1])))


  # Clean and unique initial guesses, ensure sorted
  initial_guesses_list = [np.array(sorted(g)) for g in initial_guesses_list if len(g) == m]
  # Use set to remove duplicates (convert to tuple for hashability)
  unique_guesses_tuples = set(tuple(g.tolist()) for g in initial_guesses_list)
  initial_guesses_queue = [np.array(g) for g in unique_guesses_tuples]

  # Prioritize known good roots if they are in the initial guesses
  if m in known_good_roots:
      known_good = np.sort(known_good_roots[m])
      if any(np.array_equal(guess, known_good) for guess in initial_guesses_queue):
          # Move the known good guess to the front of the queue
          initial_guesses_queue.insert(0, initial_guesses_queue.pop([i for i, guess in enumerate(initial_guesses_queue) if np.array_equal(guess, known_good)][0]))




  while time.time() - start_time < 985 and len(initial_guesses_queue) > 0:

      initial_guess = initial_guesses_queue.pop(0)
      initial_guess = np.clip(initial_guess, bounds[0][0], bounds[0][1])

      # Check remaining time before starting a new optimization run
      if time.time() - start_time >= 985:
          break

      try:
          # Slightly increase max iterations per attempt
          max_iter_per_attempt = max(400, 60 * m)
          if m > 15: max_iter_per_attempt = max(300, 40 * m)
          if m > 20: max_iter_per_attempt = max(200, 30 * m)

          # Add an absolute function evaluation limit as well, capped globally
          max_fevals_per_attempt_nm = min(max_iter_per_attempt * 1.5, 80000 + m * 2000) # Increased cap with m

          # Run Nelder-Mead first
          nm_result = scipy.optimize.minimize(
              objective_function,
              initial_guess,
              method='Nelder-Mead',
              bounds=bounds,
              options={
                  'maxiter': max_iter_per_attempt,
                  'maxfev': max_fevals_per_attempt_nm,
                  'disp': False
                  }
          )

          nm_current_roots = nm_result.x
          nm_current_roots = np.clip(nm_current_roots, bounds[0][0], bounds[0][1])
          nm_current_score = get_score(nm_current_roots)

          current_best_roots = nm_current_roots
          current_best_score = nm_current_score

          if current_best_score is not None and current_best_score > -1e9:
               if current_best_score > best_score:
                  # Found a new best score with NM or DE
                  best_score = current_best_score
                  best_roots = current_best_roots.tolist()
                  # Add to global best roots list (keeping it relatively small and diverse)
                  global_best_roots_list.append(best_roots)
                  # Keep the global_best_roots_list sorted by score and limit its size
                  global_best_roots_list = sorted(global_best_roots_list, key=lambda x: get_score(x) if get_score(x) is not None else -float('inf'), reverse=True)
                  global_best_roots_list = global_best_roots_list[:max(5, m)] # Limit size

                  # print(f"Found new best score (NM): {best_score:.4f} for m={m}")

               # Warm-starting for Nelder-Mead perturbations (slightly reduced to make room for global best exploration)
               if current_best_score is not None and current_best_score > -5.0 and time.time() - start_time < 980 and len(initial_guesses_queue) < max(15, m * 1.5):
                   perturb_scale = 0.8 + 2.5 * (m/20.0)
                   num_perturbations = 2 # Increased perturbations slightly

                   for _ in range(num_perturbations):
                       # Normal perturbation
                       perturbed_guess_normal = np.array(current_best_roots) + np.random.normal(0, perturb_scale, m)
                       initial_guesses_queue.append(np.clip(perturbed_guess_normal, bounds[0][0], bounds[0][1]))

                       if m > 1:
                            # Swap perturbation
                            perturbed_guess_swap = np.array(current_best_roots)
                            idx1, idx2 = np.random.choice(m, 2, replace=False)
                            perturbed_guess_swap[[idx1, idx2]] = perturbed_guess_swap[[idx2, idx1]]
                            initial_guesses_queue.append(np.clip(perturbed_guess_swap, bounds[0][0], bounds[0][1]))

                            # Perturb close roots
                            sorted_roots = np.sort(current_best_roots)
                            diffs = np.diff(sorted_roots)
                            close_pairs_indices = np.where(diffs < np.mean(diffs) / 2.0)[0] # Threshold for closeness
                            if len(close_pairs_indices) > 0:
                                chosen_pair_index = np.random.choice(close_pairs_indices)
                                root1_idx = chosen_pair_index
                                root2_idx = chosen_pair_index + 1
                                perturbed_guess_close = np.copy(current_best_roots)
                                # Find the indices of the roots in the original (unsorted) array
                                root1_val = sorted_roots[root1_idx]
                                root2_val = sorted_roots[root2_idx]
                                original_indices_1 = np.where(np.isclose(current_best_roots, root1_val))[0]
                                original_indices_2 = np.where(np.isclose(current_best_roots, root2_val))[0]

                                if original_indices_1.size > 0 and original_indices_2.size > 0:
                                    idx1 = np.random.choice(original_indices_1)
                                    idx2 = np.random.choice(original_indices_2)
                                    # Apply a small, opposite perturbation to the close roots
                                    perturb_amount = np.random.uniform(0.1, 0.5) * diffs[chosen_pair_index]
                                    perturbed_guess_close[idx1] += perturb_amount
                                    perturbed_guess_close[idx2] -= perturb_amount
                                    initial_guesses_queue.append(np.clip(perturbed_guess_close, bounds[0][0], bounds[0][1]))
          #Differential evolution is already handled inside
          if remaining_time > 15 and current_best_score is not None and current_best_score > -1e9: # Only run DE if NM was somewhat successful
              # Allocate a portion of remaining time to DE, cap it to prevent excessively long runs
              # Make DE time limit more dynamic based on remaining time
              de_time_limit = min(remaining_time * 0.8, 400 + m * 8 + remaining_time * 0.1) # Increased cap and added dependency on remaining time

              # If NM found a better point than the initial guess, start DE from there, or use a mix of best roots
              de_initial_population = []
              if current_best_score is not None and current_best_score > -1e8 and (objective_function(current_best_roots) < objective_function(initial_guess) or current_best_score > get_score(initial_guess)):
                  de_initial_population.append(current_best_roots)
              # Add some of the global best roots to the initial DE population (ensure diversity)
              for roots in global_best_roots_list[:min(5, len(global_best_roots_list))]:
                  de_initial_population.append(np.array(roots))


              try:
                 # Adjust DE parameters based on remaining time
                 dynamic_maxiter = min(1500, 60 * m + int(remaining_time * 0.5)) # More iterations if time permits
                 dynamic_popsize = max(20, m * 2.5 + int(remaining_time * 0.1)) # Larger population if time permits

                 de_result = scipy.optimize.differential_evolution( # Explicitly call from scipy.optimize
                     objective_function,
                     bounds=bounds,
                     popsize=dynamic_popsize,
                     maxiter=dynamic_maxiter,
                     tol=0.005, # Increased tolerance for potentially better convergence
                     disp=False,
                     # seed=42 # Optional: for reproducibility
                     # Use initial population if available, otherwise let DE generate random
                     init=de_initial_population if de_initial_population else "random"
                 )

                 de_current_roots = de_result.x
                 de_current_roots = np.clip(de_current_roots, bounds[0][0], bounds[0][1])
                 de_current_score = get_score(de_current_roots)

                 if de_current_score is not None and de_current_score > -1e9:
                     if de_current_score > best_score:
                         best_score = de_current_score
                         best_roots = de_current_roots.tolist()
                         # Add to global best roots list
                         global_best_roots_list.append((best_roots, current_best_score))
                         global_best_roots_list = sorted(global_best_roots_list, key=lambda x: x[1], reverse=True)
                         global_best_roots_list = global_best_roots_list[:max(5, m)] # Limit size

                         # print(f"Found new best score (DE): {best_score:.4f} for m={m}")

              except Exception as e:
                   # print(f"Differential Evolution failed with error {e}")
                   pass


      except Exception as e:
          # print(f"Optimization attempt failed for guess {initial_guess[:5]}... with error {e}")
          pass

  # If no valid roots were found during optimization, try the initial guesses
  if best_roots is None or best_score < -1e8:
      fallback_score = -float('inf')
      fallback_roots = None
      for guess in initial_guesses_list:
          try:
              score = get_score(guess)
              if score is not None and score > -1e9:
                  if score > fallback_score:
                      fallback_score = score
                      fallback_roots = guess.tolist()
          except Exception:
              pass

      if fallback_roots is not None:
          return fallback_roots
      else:
          # Return a default guess if all else fails
          if m in known_good_roots:
              return known_good_roots[m].tolist()
          else:
              # Fallback to a simple structured guess or random
              if m >= 2:
                  start_val = 35.0
                  end_val = 50.0 + (m-6)*2
                  if m-2 >= 1:
                       roots = [4.0, 5.0] + list(np.linspace(start_val, end_val, m - 2))
                  else: # m=2
                       roots = [4.0, 5.0]
              elif m == 1:
                   roots = [40.0]
              else:
                   roots = np.random.uniform(1.0, 100.0, m).tolist() # Safeguard

              return np.clip(np.array(roots), bounds[0][0], bounds[0][1]).tolist()


  return best_roots
